In [11]:
# If true, the WAV files will be read and their features will be saved in the CSV files
# As this is the most time consuming task, only enable it if you don't have the CSV files yet
CREATE_CSV_FILES = True

In [12]:
# Defines the names of the CSV files
TRAIN_CSV_FILE = "train.csv"
TEST_CSV_FILE = "test.csv"
MORE_TRAIN_CSV_FILE = "more_train.csv"
MORE_TEST_CSV_FILE = "more_test.csv"


In [72]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import librosa
import csv
import os
import chromaFeatures 
import librosa.display
if(os.path.exists(TRAIN_CSV_FILE) and os.path.isfile(TRAIN_CSV_FILE)):
    os.remove(TRAIN_CSV_FILE)
if(os.path.exists(TEST_CSV_FILE) and os.path.isfile(TEST_CSV_FILE)):
    os.remove(TEST_CSV_FILE)
def extractWavFeatures(soundFilesFolder, csvFileName,label):
    print("The features of the files in the folder "+soundFilesFolder+" will be saved to "+csvFileName)
    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate '
    # header = 'filename '
    for i in range(1, 21):
        header += f'mfcc{i} '
    header += 'label '
    header = header.split()
    if not os.path.exists(csvFileName):
        file = open(csvFileName, 'a', newline='')
        writer = csv.writer(file)
        writer.writerow(header)
    else:
        file = open(csvFileName, 'a', newline='')
        writer = csv.writer(file)
    genres = '1 2 3 4 5 6 7 8 9 0'.split()
    for filename in os.listdir(soundFilesFolder):
        number = f'{soundFilesFolder}/{filename}'
        y, sr = librosa.load(number, mono=True, duration=30)
        # remove leading and trailing silence
        y, index = librosa.effects.trim(y)
        # chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_stft = chromaFeatures.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append+=f' {label}'
        writer.writerow(to_append.split())
        # writer.writerow(to_append.split())
    file.close()

if (CREATE_CSV_FILES == True):
    extractWavFeatures("recordings/close", TRAIN_CSV_FILE,1)
    extractWavFeatures("recordings1/open", TRAIN_CSV_FILE,0)
    extractWavFeatures("recordings1/close", TRAIN_CSV_FILE,1)
    extractWavFeatures("recordings/open", TRAIN_CSV_FILE,0)
    # extractWavFeatures("recordings/others", TRAIN_CSV_FILE,2)
    # extractWavFeatures("recordings/open", TEST_CSV_FILE,0)
    print("CSV files are created")
else:
    print("CSV files creation is skipped")


The features of the files in the folder recordings/close will be saved to train.csv
The features of the files in the folder recordings1/open will be saved to train.csv
The features of the files in the folder recordings1/close will be saved to train.csv
The features of the files in the folder recordings/open will be saved to train.csv
CSV files are created


In [14]:
# melspec_mean_list=[]
# melspec_var_list=[]
# for i in range(1,16):
#     y, sr = librosa.load(f'..\\data\\recordings\\Dina\\open\\h_open_({i}).wav', mono=True, duration=30)
#     # this is the number of samples in a window per fft
#     # n_fft = 2048
#     # # The amount of samples we are shifting after each fft
#     # hop_length = 512
#     mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
#     melspec_mean = np.mean(mel_signal)
#     melspec_var = np.var(mel_signal)
#     melspec_mean_list.append(melspec_mean)
#     melspec_var_list.append(melspec_var)
#     # print(f'mean{i}:{melspec_mean}')
#     # print(f'var{i}:{melspec_var}')
#     # spectrogram = np.abs(mel_signal)
#     # power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
#     # librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma', hop_length=hop_length)
# print(f'mean_max{max(melspec_mean_list)},mean_min:{min(melspec_mean_list)},mean_mean{np.mean(melspec_mean_list)}')
# print(f'var_max{max(melspec_var_list)},var_min:{min(melspec_var_list)},var_mean{np.mean(melspec_var_list)}')

In [15]:
# # melspec_mean_list=[]
# # melspec_var_list=[]
# # for i in range(1,16):
# y, sr = librosa.load(f'..\\data\\recordings\\Mariam\\close\\b_close_(1).wav', mono=True, duration=30)
# # this is the number of samples in a window per fft
# # n_fft = 2048
# # # The amount of samples we are shifting after each fft
# # hop_length = 512
# mel_signal = librosa.feature.melspectrogram(y=y, sr=sr)
# melspec_mean = np.mean(mel_signal)
# melspec_var = np.var(mel_signal)
# melspec_mean_list.append(melspec_mean)
# melspec_var_list.append(melspec_var)
# # print(f'mean{i}:{melspec_mean}')
# # print(f'var{i}:{melspec_var}')
# spectrogram = np.abs(mel_signal)
# power_to_db = librosa.power_to_db(spectrogram, ref=np.max)
# librosa.display.specshow(power_to_db, sr=sr, x_axis='time', y_axis='mel', cmap='magma', hop_length=hop_length)
# # print(f'mean_max{max(melspec_mean_list)},mean_min:{min(melspec_mean_list)},mean_mean{np.mean(melspec_mean_list)}')
# # print(f'var_max{max(melspec_var_list)},var_min:{min(melspec_var_list)},var_mean{np.mean(melspec_var_list)}')

In [16]:
# y, sr = librosa.load('..\\recordings\\test\\close(13).wav', mono=True, duration=30)
# mfcc = librosa.feature.mfcc(y=y, sr=sr)
# mfcc
# mfcc_list=[]
# for e in mfcc:
#             mfcc_list.append(e)
# print(len(mfcc_list))
# print(mfcc_list)
# # mfcc.shape



# import freature_ex as ex

# mfcc=ex.get_mfcc(y,sr)
# mfcc_list=[]
# for e in mfcc:
#             mfcc_list.append(e)
# print(len(mfcc_list))
# print(mfcc_list)

In [17]:
# y, sr = librosa.load('..\\recordings\\test\\close(13).wav', mono=True, duration=30)
# op=np.mean(librosa.feature.chroma_stft(y=y,sr=sr))
# op

In [18]:
# import chromaFeatures as ft 
# op= np.mean(ft.chroma_stft(y=y,sr=sr))
# op

In [19]:
#Reading a dataset and convert file name to corresponding number

import pandas as pd
import csv
from sklearn import preprocessing

def preProcessData(csvFileName):
    print(csvFileName+ " will be preprocessed")
    data = pd.read_csv(csvFileName, error_bad_lines=False)
    # data['number'] = data['filename'].str[:1]
    #Dropping unnecessary columns
    data = data.drop(['filename'],axis=1)
    # data = data.drop(['label'],axis=1)
    # data = data.drop(['chroma_stft'],axis=1)
    data.shape

    print("Preprocessing is finished")
    print(data.head())
    return data

trainData = preProcessData(TRAIN_CSV_FILE)
# testData = preProcessData(TEST_CSV_FILE)
# moreTrainData = preProcessData(MORE_TRAIN_CSV_FILE)
# moreTestData = preProcessData(MORE_TEST_CSV_FILE)



train.csv will be preprocessed
Preprocessing is finished
   chroma_stft      rmse  spectral_centroid  spectral_bandwidth      rolloff  \
0     0.233211  0.060876        1980.499957         2307.433957  4949.485518   
1     0.285432  0.035748        1823.559789         2154.511190  4440.733754   
2     0.278946  0.048961        1876.957824         2283.669924  5114.660942   
3     0.263745  0.055076        1800.239710         2194.800112  4725.581055   
4     0.246478  0.058140        1536.114708         2002.493192  3721.329013   

   zero_crossing_rate       mfcc1       mfcc2      mfcc3      mfcc4  ...  \
0            0.074790 -326.950043  112.391731  24.532602  -7.646422  ...   
1            0.074962 -384.217377  113.313568  24.379700  -2.288423  ...   
2            0.078232 -328.949310  120.625397  11.633698 -10.470407  ...   
3            0.072797 -316.711884  127.934776  23.606880 -16.524025  ...   
4            0.057684 -324.862000  134.351212  20.889097 -10.809068  ...   

     

C:\Users\hp\AppData\Local\Temp\ipykernel_30432\3181961187.py:9: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(csvFileName, error_bad_lines=False)


## Section 2

There are 50 recordings for each digit for each speaker: Jackson, Nicolas and Theo (total 1500 recordings)

Training data has 49 recordings for each digit for each speaker: 1470 recordings total.
Test data has 1 recordings for each digit for each speaker: 30 recordings total.

The data used here comes from the recordings stored in:
* ../data/recordings/train
* ../data/recordings/test

The model will be trained to predict the spoken digit.

In [20]:
# Splitting the dataset into training, validation and testing dataset
from sklearn.model_selection import train_test_split
X = np.array(trainData.iloc[:, :-1], dtype = float)
y = trainData.iloc[:, -1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# X_test = np.array(testData.iloc[:, :-1], dtype = float)
# y_test = testData.iloc[:, -1]

print("Y from training data:", y_train.shape)
print("Y from validation data:", y_val.shape)
# print("Y from test data:", y_test.shape)


Y from training data: (128,)
Y from validation data: (32,)


In [21]:
# #Normalizing the dataset
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# scaler = StandardScaler()
# X_train = scaler.fit_transform( X_train )
# X_val = scaler.transform( X_val )
# X_test = scaler.transform( X_test )

# print("X from training data", X_train.shape)
# print("X from validation data", X_val.shape)
# print("X from test data", X_test.shape)


In [22]:
# import SVC classifier
from sklearn.svm import SVC


# import metrics to compute accuracy
from sklearn.metrics import accuracy_score


# instantiate classifier with default hyperparameters
svc=SVC() 


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_val)


# compute and print accuracy score
print('Model accuracy score with default hyperparameters: {0:0.4f}'. format(accuracy_score(y_val, y_pred)))

Model accuracy score with default hyperparameters: 0.8125


In [23]:
# instantiate classifier with rbf kernel and C=100
svc=SVC(C=10000.0) 


# fit classifier to training set
svc.fit(X_train,y_train)


# make predictions on test set
y_pred=svc.predict(X_val)


# compute and print accuracy score
print('Model accuracy score with rbf kernel and C=100.0 : {0:0.4f}'. format(accuracy_score(y_val, y_pred)))

Model accuracy score with rbf kernel and C=100.0 : 0.9688


In [24]:
# instantiate classifier with linear kernel and C=1.0
poly_svc=SVC(kernel='poly', C=1000.0) 


# fit classifier to training set
poly_svc.fit(X_train,y_train)


# make predictions on test set
y_pred_test=poly_svc.predict(X_val)


# compute and print accuracy score
print('Model accuracy score with linear kernel and C=1.0 : {0:0.4f}'. format(accuracy_score(y_val, y_pred_test)))
print("Accuracy on training set: {:.3f}".format(poly_svc.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(poly_svc.score(X_val, y_val)))

Model accuracy score with linear kernel and C=1.0 : 0.8750
Accuracy on training set: 0.797
Accuracy on test set: 0.875


In [25]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=5, random_state=0).fit(X_train, y_train)
print("\nRandom Forests")
print("Accuracy on training set: {:.3f}".format(forest.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(X_val, y_val)))


Random Forests
Accuracy on training set: 0.977
Accuracy on test set: 0.875


In [26]:
def extractWavFeatures():
    list_of_features=[]
    y, sr = librosa.load('../audio/audio.wav', mono=True, duration=30)
    # remove leading and trailing silence
    y, index = librosa.effects.trim(y)

    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    list_of_features.append(np.mean(chroma_stft))
    list_of_features.append(np.mean(rmse))
    list_of_features.append(np.mean(spec_cent))
    list_of_features.append(np.mean(spec_bw))
    list_of_features.append(np.mean(rolloff))
    list_of_features.append(np.mean(zcr))

    for e in mfcc:
            list_of_features.append(np.mean(e))
    
    return(list_of_features)

In [27]:
speech_features=[]
speech_features.append(extractWavFeatures())

In [28]:
forest.predict(speech_features)

array([0], dtype=int64)

In [29]:
svc.predict(speech_features)

array([0], dtype=int64)

In [30]:
from sklearn.tree import DecisionTreeClassifier
#Train decision tree model
tree = DecisionTreeClassifier(random_state=1).fit(X_train, y_train)
print("\nDecision Tree")
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_val, y_val)))


Decision Tree
Accuracy on training set: 1.000
Accuracy on test set: 0.844


In [61]:
import pickle 
pickle.dump(tree ,open('../trainedModel.sav' , 'wb'))
model= pickle.load(open('../trainedModel.sav' , 'rb'))

In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
gmm=GaussianMixture(n_components=5)
gmm.fit(X_train,y_train)
ygmm_pred_class = gmm.predict(X_val)
print(accuracy_score(y_val, ygmm_pred_class))
print(confusion_matrix(y_val, ygmm_pred_class))

0.0
[[ 0  0  6 10]
 [ 0  0  0 16]
 [ 0  0  0  0]
 [ 0  0  0  0]]


In [33]:
# instantiate classifier with linear kernel and C=1.0
rbf_svc=SVC(kernel='rbf', C=1000.0) 


# fit classifier to training set
rbf_svc.fit(X_train,y_train)


# make predictions on test set
y_pred_test=rbf_svc.predict(X_val)


# compute and print accuracy score
print('Model accuracy score with linear kernel and C=1.0 : {0:0.4f}'. format(accuracy_score(y_val, y_pred_test)))
print("Accuracy on training set: {:.3f}".format(rbf_svc.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(rbf_svc.score(X_val, y_val)))

Model accuracy score with linear kernel and C=1.0 : 0.9062
Accuracy on training set: 0.867
Accuracy on test set: 0.906


In [77]:
def extractWavFeatures():
    list_of_features=[]
    y, sr = librosa.load('../audio/audio.wav', mono=True, duration=30)
    # remove leading and trailing silence
    y, index = librosa.effects.trim(y)

    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    list_of_features.append(np.mean(chroma_stft))
    list_of_features.append(np.mean(rmse))
    list_of_features.append(np.mean(spec_cent))
    list_of_features.append(np.mean(spec_bw))
    list_of_features.append(np.mean(rolloff))
    list_of_features.append(np.mean(zcr))

    for e in mfcc:
            list_of_features.append(np.mean(e))
    
    return(list_of_features)
speech_features=[]
speech_features.append(extractWavFeatures())
print(forest.predict(speech_features))
print(svc.predict(speech_features))
print(tree.predict(speech_features))
print(poly_svc.predict(speech_features))
print(rbf_svc.predict(speech_features))
# print(gmm.predict(speech_features))

[0]
[0]
[0]
[0]
[0]
